In [ ]:
!nvidia-smi | head

Tue Sep 17 00:58:25 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   61C    P0              30W /  70W |   7641MiB / 15360MiB |      0%      Default |


In [ ]:
!lscpu | grep -E "Model|Socket"

Model name:                           Intel(R) Xeon(R) CPU @ 2.30GHz
Model:                                63
Socket(s):                            1


# Setup

In [ ]:
!pip install -U polars[gpu] --extra-index-url=https://pypi.nvidia.com

In [ ]:
!pip install hvplot jupyter_bokeh holoviews==1.19 pynvml

In [ ]:
import pynvml
pynvml.nvmlInit()
pynvml.nvmlDeviceGetName(pynvml.nvmlDeviceGetHandleByIndex(0))
mem = pynvml.nvmlDeviceGetMemoryInfo(pynvml.nvmlDeviceGetHandleByIndex(0))
mem = mem.total/1e9
if mem < 24:
    !wget https://storage.googleapis.com/rapidsai/polars-demo/transactions-t4-20.parquet -O transactions.parquet
else:
    !wget https://storage.googleapis.com/rapidsai/polars-demo/transactions.parquet -O transactions.parquet

!wget https://storage.googleapis.com/rapidsai/polars-demo/rainfall_data_2010_2020.csv

--2024-09-17 00:16:22--  https://storage.googleapis.com/rapidsai/polars-demo/transactions-t4-20.parquet
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.196.207, 74.125.141.207, 173.194.210.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.196.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 857658422 (818M) [application/octet-stream]
Saving to: ‘transactions.parquet’

transactions.parque 100%[===================>] 817.93M  61.2MB/s    in 12s     

2024-09-17 00:16:35 (66.4 MB/s) - ‘transactions.parquet’ saved [857658422/857658422]

--2024-09-17 00:16:35--  https://storage.googleapis.com/rapidsai/polars-demo/rainfall_data_2010_2020.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.196.207, 74.125.141.207, 173.194.210.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.196.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 131421 (128K) [

# Getting Started

In [ ]:
import polars as pl
import hvplot.polars # Using the hvplot as the plotting backend
from polars.testing import assert_frame_equal

In [ ]:
pl.__version__ # Make sure the version is >= `1.6.0`; If not restart the session.

'1.7.1'

In [ ]:
transactions = pl.scan_parquet("transactions.parquet")

In [ ]:
transactions.collect_schema()

Schema([('CUST_ID', String),
        ('START_DATE', Date),
        ('END_DATE', Date),
        ('TRANS_ID', String),
        ('DATE', Date),
        ('YEAR', Int64),
        ('MONTH', Int64),
        ('DAY', Int64),
        ('EXP_TYPE', String),
        ('AMOUNT', Float64)])

In [ ]:
transactions.head(5).collect()

shape: (5, 10)
┌────────────┬────────────┬──────────┬─────────────────┬───┬───────┬─────┬───────────────┬────────┐
│ CUST_ID    ┆ START_DATE ┆ END_DATE ┆ TRANS_ID        ┆ … ┆ MONTH ┆ DAY ┆ EXP_TYPE      ┆ AMOUNT │
│ ---        ┆ ---        ┆ ---      ┆ ---             ┆   ┆ ---   ┆ --- ┆ ---           ┆ ---    │
│ str        ┆ date       ┆ date     ┆ str             ┆   ┆ i64   ┆ i64 ┆ str           ┆ f64    │
╞════════════╪════════════╪══════════╪═════════════════╪═══╪═══════╪═════╪═══════════════╪════════╡
│ CI6XLYUMQK ┆ 2015-05-01 ┆ null     ┆ T8I9ZB5A6X90UG8 ┆ … ┆ 9     ┆ 11  ┆ Motor/Travel  ┆ 20.27  │
│ CI6XLYUMQK ┆ 2015-05-01 ┆ null     ┆ TZ4JSLS7SC7FO9H ┆ … ┆ 2     ┆ 8   ┆ Motor/Travel  ┆ 12.85  │
│ CI6XLYUMQK ┆ 2015-05-01 ┆ null     ┆ TTUKRDDJ6B6F42H ┆ … ┆ 8     ┆ 1   ┆ Housing       ┆ 383.8  │
│ CI6XLYUMQK ┆ 2015-05-01 ┆ null     ┆ TDUHFRUKGPPI6HD ┆ … ┆ 3     ┆ 16  ┆ Entertainment ┆ 5.72   │
│ CI6XLYUMQK ┆ 2015-05-01 ┆ null     ┆ T0JBZHBMSVRFMMD ┆ … ┆ 5     ┆ 15  ┆ Entertainment ┆ 11.06  │
└────────────┴────────────┴──────────┴─────────────────┴───┴───────┴─────┴───────────────┴────────┘

In [ ]:
transactions.select(pl.col("AMOUNT").sum()).collect()

shape: (1, 1)
┌──────────┐
│ AMOUNT   │
│ ---      │
│ f64      │
╞══════════╡
│ 3.6183e9 │
└──────────┘

In [ ]:
gpu_engine = pl.GPUEngine(
    device=0, # This is the default
    raise_on_fail=True, # Fail loudly if we can't run on the GPU.
)

In [ ]:
transactions.select(pl.col("AMOUNT").sum()).collect(engine=gpu_engine)

shape: (1, 1)
┌──────────┐
│ AMOUNT   │
│ ---      │
│ f64      │
╞══════════╡
│ 3.6183e9 │
└──────────┘

In [ ]:
%%time

res_cpu = (
    transactions
    .group_by("CUST_ID")
    .agg(pl.col("AMOUNT").sum())
    .sort(by="AMOUNT", descending=True)
    .head()
    .collect()
)
res_cpu

CPU times: user 5.06 s, sys: 3.83 s, total: 8.89 s
Wall time: 7.22 s


shape: (5, 2)
┌────────────┬──────────┐
│ CUST_ID    ┆ AMOUNT   │
│ ---        ┆ ---      │
│ str        ┆ f64      │
╞════════════╪══════════╡
│ CA9UYOQ5DA ┆ 2.0290e6 │
│ CJUK2MTM5Q ┆ 1.8115e6 │
│ CYXX1NBIKL ┆ 1.8082e6 │
│ C6ILEYAYQ9 ┆ 1.7961e6 │
│ CCNBC305GI ┆ 1.7274e6 │
└────────────┴──────────┘

In [ ]:
%%time

res_gpu = (
    transactions
    .group_by("CUST_ID")
    .agg(pl.col("AMOUNT").sum())
    .sort(by="AMOUNT", descending=True)
    .head()
    .collect(engine=gpu_engine)
)
res_gpu

CPU times: user 374 ms, sys: 140 ms, total: 514 ms
Wall time: 497 ms


shape: (5, 2)
┌────────────┬──────────┐
│ CUST_ID    ┆ AMOUNT   │
│ ---        ┆ ---      │
│ str        ┆ f64      │
╞════════════╪══════════╡
│ CA9UYOQ5DA ┆ 2.0290e6 │
│ CJUK2MTM5Q ┆ 1.8115e6 │
│ CYXX1NBIKL ┆ 1.8082e6 │
│ C6ILEYAYQ9 ┆ 1.7961e6 │
│ CCNBC305GI ┆ 1.7274e6 │
└────────────┴──────────┘

In [ ]:
assert_frame_equal(res_cpu, res_gpu)

In [ ]:
query = """
SELECT CUST_ID, SUM(AMOUNT) as sum_amt
FROM transactions
GROUP BY CUST_ID
ORDER BY sum_amt desc
LIMIT 5
"""

%time pl.sql(query).collect()
%time pl.sql(query).collect(engine=gpu_engine)

CPU times: user 4.97 s, sys: 2.93 s, total: 7.9 s
Wall time: 5.27 s
CPU times: user 307 ms, sys: 64.9 ms, total: 372 ms
Wall time: 331 ms


shape: (5, 2)
┌────────────┬──────────┐
│ CUST_ID    ┆ sum_amt  │
│ ---        ┆ ---      │
│ str        ┆ f64      │
╞════════════╪══════════╡
│ CA9UYOQ5DA ┆ 2.0290e6 │
│ CJUK2MTM5Q ┆ 1.8115e6 │
│ CYXX1NBIKL ┆ 1.8082e6 │
│ C6ILEYAYQ9 ┆ 1.7961e6 │
│ CCNBC305GI ┆ 1.7274e6 │
└────────────┴──────────┘

In [ ]:
%%time

(
    transactions
    .group_by("CUST_ID")
    .agg(pl.col("AMOUNT").max().alias("max_amount"))
    .sort(by="max_amount", descending=True)
    .head()
    .collect()
)

CPU times: user 5.26 s, sys: 3.12 s, total: 8.39 s
Wall time: 6.81 s


shape: (5, 2)
┌────────────┬────────────┐
│ CUST_ID    ┆ max_amount │
│ ---        ┆ ---        │
│ str        ┆ f64        │
╞════════════╪════════════╡
│ CIP0I11MG2 ┆ 6201.45    │
│ C4O38N5TQS ┆ 6077.49    │
│ CL2M3N3K90 ┆ 6041.59    │
│ CC472PU9O8 ┆ 5929.14    │
│ CGR8UI27OK ┆ 5903.61    │
└────────────┴────────────┘

In [ ]:
%%time

(
    transactions
    .group_by("CUST_ID")
    .agg(pl.col("AMOUNT").max().alias("max_amount"))
    .sort(by="max_amount", descending=True)
    .head()
    .collect(engine=gpu_engine)
)

CPU times: user 319 ms, sys: 57.7 ms, total: 376 ms
Wall time: 372 ms


shape: (5, 2)
┌────────────┬────────────┐
│ CUST_ID    ┆ max_amount │
│ ---        ┆ ---        │
│ str        ┆ f64        │
╞════════════╪════════════╡
│ CIP0I11MG2 ┆ 6201.45    │
│ C4O38N5TQS ┆ 6077.49    │
│ CL2M3N3K90 ┆ 6041.59    │
│ CC472PU9O8 ┆ 5929.14    │
│ CGR8UI27OK ┆ 5903.61    │
└────────────┴────────────┘

In [ ]:
%%time

(
    transactions
    .filter(pl.col("CUST_ID") == "CIP0I11MG2")
    .select(pl.col("AMOUNT").max())
    .collect()
)

CPU times: user 2.03 s, sys: 205 ms, total: 2.24 s
Wall time: 1.21 s


shape: (1, 1)
┌─────────┐
│ AMOUNT  │
│ ---     │
│ f64     │
╞═════════╡
│ 6201.45 │
└─────────┘

In [ ]:
%%time

(
    transactions
    .filter(pl.col("CUST_ID") == "CIP0I11MG2")
    .select(pl.col("AMOUNT").max())
    .collect(engine=gpu_engine)
)

CPU times: user 220 ms, sys: 51 ms, total: 271 ms
Wall time: 266 ms


shape: (1, 1)
┌─────────┐
│ AMOUNT  │
│ ---     │
│ f64     │
╞═════════╡
│ 6201.45 │
└─────────┘

In [ ]:
res, prof = (
    transactions
    .filter(pl.col("CUST_ID") == "CIP0I11MG2")
    .select(pl.col("AMOUNT").max())
    .profile()
)

prof.with_columns(
    ((pl.col("end") - pl.col("start")) / pl.col("end").max() * 100)
    .alias("pct_time_spent")
)

shape: (3, 4)
┌─────────────────────────────────┬─────────┬─────────┬────────────────┐
│ node                            ┆ start   ┆ end     ┆ pct_time_spent │
│ ---                             ┆ ---     ┆ ---     ┆ ---            │
│ str                             ┆ u64     ┆ u64     ┆ f64            │
╞═════════════════════════════════╪═════════╪═════════╪════════════════╡
│ optimization                    ┆ 0       ┆ 7       ┆ 0.000598       │
│ parquet(transactions.parquet, … ┆ 7       ┆ 1171036 ┆ 99.995303      │
│ select(AMOUNT)                  ┆ 1171046 ┆ 1171084 ┆ 0.003245       │
└─────────────────────────────────┴─────────┴─────────┴────────────────┘

In [ ]:
%%time

res = (
    transactions
    .group_by(["EXP_TYPE", "YEAR", "MONTH"])
    .agg(pl.mean("AMOUNT"))
    .sort(["EXP_TYPE", "YEAR", "MONTH"])
    .collect()
)

CPU times: user 13.8 s, sys: 8.53 s, total: 22.3 s
Wall time: 15.7 s


In [ ]:
%%time

res = (
    transactions
    .group_by(["EXP_TYPE", "YEAR", "MONTH"])
    .agg(pl.mean("AMOUNT"))
    .sort(["EXP_TYPE", "YEAR", "MONTH"])
    .collect(engine=gpu_engine)
)

CPU times: user 410 ms, sys: 146 ms, total: 556 ms
Wall time: 495 ms


In [ ]:
res = res.with_columns(
    pl.datetime(pl.col("YEAR"), pl.col("MONTH"), day=1)
    .alias("year-mon")
)

res.hvplot.scatter(x="year-mon", y="AMOUNT", by="EXP_TYPE")

:NdOverlay   [EXP_TYPE]
   :Scatter   [year-mon]   (AMOUNT)

In [ ]:
res

shape: (1_584, 5)
┌─────────────────────┬──────┬───────┬────────────┬─────────────────────┐
│ EXP_TYPE            ┆ YEAR ┆ MONTH ┆ AMOUNT     ┆ year-mon            │
│ ---                 ┆ ---  ┆ ---   ┆ ---        ┆ ---                 │
│ str                 ┆ i64  ┆ i64   ┆ f64        ┆ datetime[μs]        │
╞═════════════════════╪══════╪═══════╪════════════╪═════════════════════╡
│ Bills and Utilities ┆ 2010 ┆ 1     ┆ 85.415487  ┆ 2010-01-01 00:00:00 │
│ Bills and Utilities ┆ 2010 ┆ 2     ┆ 90.495608  ┆ 2010-02-01 00:00:00 │
│ Bills and Utilities ┆ 2010 ┆ 3     ┆ 94.857936  ┆ 2010-03-01 00:00:00 │
│ Bills and Utilities ┆ 2010 ┆ 4     ┆ 95.238054  ┆ 2010-04-01 00:00:00 │
│ Bills and Utilities ┆ 2010 ┆ 5     ┆ 94.464738  ┆ 2010-05-01 00:00:00 │
│ …                   ┆ …    ┆ …     ┆ …          ┆ …                   │
│ Tax                 ┆ 2020 ┆ 8     ┆ 233.777692 ┆ 2020-08-01 00:00:00 │
│ Tax                 ┆ 2020 ┆ 9     ┆ 234.608419 ┆ 2020-09-01 00:00:00 │
│ Tax                 ┆ 2020 ┆ 10    ┆ 234.351233 ┆ 2020-10-01 00:00:00 │
│ Tax                 ┆ 2020 ┆ 11    ┆ 234.432279 ┆ 2020-11-01 00:00:00 │
│ Tax                 ┆ 2020 ┆ 12    ┆ 234.278336 ┆ 2020-12-01 00:00:00 │
└─────────────────────┴──────┴───────┴────────────┴─────────────────────┘

In [ ]:
names = ['Location', 'Rainfall (inches)', 'Date', 'YEAR', 'MONTH', 'DAY']

weather = pl.scan_csv("rainfall_data_2010_2020.csv", new_columns=names)
weather.head().collect()

shape: (5, 6)
┌──────────┬───────────────────┬──────────┬──────┬───────┬─────┐
│ Location ┆ Rainfall (inches) ┆ Date     ┆ YEAR ┆ MONTH ┆ DAY │
│ ---      ┆ ---               ┆ ---      ┆ ---  ┆ ---   ┆ --- │
│ str      ┆ f64               ┆ i64      ┆ i64  ┆ i64   ┆ i64 │
╞══════════╪═══════════════════╪══════════╪══════╪═══════╪═════╡
│ Tatooine ┆ 0.33              ┆ 20100101 ┆ 2010 ┆ 1     ┆ 1   │
│ Tatooine ┆ 0.0               ┆ 20100102 ┆ 2010 ┆ 1     ┆ 2   │
│ Tatooine ┆ 0.28              ┆ 20100103 ┆ 2010 ┆ 1     ┆ 3   │
│ Tatooine ┆ 0.26              ┆ 20100104 ┆ 2010 ┆ 1     ┆ 4   │
│ Tatooine ┆ 0.39              ┆ 20100105 ┆ 2010 ┆ 1     ┆ 5   │
└──────────┴───────────────────┴──────────┴──────┴───────┴─────┘

In [ ]:
weather_cleaned = (
    weather
    .with_columns(pl.col("Date").cast(pl.Utf8).str.strptime(pl.Date(), "%Y%m%d"))
    .collect()
)

In [ ]:
%%time

(
    transactions
    .join(
        other=weather_cleaned.lazy(),
        left_on="DATE",
        right_on="Date",
        how="inner"
    )
    .group_by(["EXP_TYPE", "DATE"])
    .agg(pl.mean("Rainfall (inches)"))
    .sort(["DATE", "EXP_TYPE", "Rainfall (inches)"])
    .head()
    .collect()
)

CPU times: user 15.8 s, sys: 4.98 s, total: 20.8 s
Wall time: 18.9 s


shape: (5, 3)
┌─────────────────────┬────────────┬───────────────────┐
│ EXP_TYPE            ┆ DATE       ┆ Rainfall (inches) │
│ ---                 ┆ ---        ┆ ---               │
│ str                 ┆ date       ┆ f64               │
╞═════════════════════╪════════════╪═══════════════════╡
│ Bills and Utilities ┆ 2010-01-01 ┆ 0.33              │
│ Clothing            ┆ 2010-01-01 ┆ 0.33              │
│ Education           ┆ 2010-01-01 ┆ 0.33              │
│ Entertainment       ┆ 2010-01-01 ┆ 0.33              │
│ Fines               ┆ 2010-01-01 ┆ 0.33              │
└─────────────────────┴────────────┴───────────────────┘

In [ ]:
%%time

(
    transactions
    .join(
        other=weather_cleaned.lazy(),
        left_on="DATE",
        right_on="Date",
        how="inner"
    )
    .group_by(["EXP_TYPE", "DATE"])
    .agg(pl.mean("Rainfall (inches)"))
    .sort(["DATE", "EXP_TYPE", "Rainfall (inches)"])
    .head()
    .collect(engine=gpu_engine)
)

CPU times: user 384 ms, sys: 35.5 ms, total: 419 ms
Wall time: 395 ms


shape: (5, 3)
┌─────────────────────┬────────────┬───────────────────┐
│ EXP_TYPE            ┆ DATE       ┆ Rainfall (inches) │
│ ---                 ┆ ---        ┆ ---               │
│ str                 ┆ date       ┆ f64               │
╞═════════════════════╪════════════╪═══════════════════╡
│ Bills and Utilities ┆ 2010-01-01 ┆ 0.33              │
│ Clothing            ┆ 2010-01-01 ┆ 0.33              │
│ Education           ┆ 2010-01-01 ┆ 0.33              │
│ Entertainment       ┆ 2010-01-01 ┆ 0.33              │
│ Fines               ┆ 2010-01-01 ┆ 0.33              │
└─────────────────────┴────────────┴───────────────────┘

In [ ]:
res = (
    transactions
    .join(
        other=weather_cleaned.lazy(),
        left_on="DATE",
        right_on="Date",
        how="inner"
    )
    .group_by(["EXP_TYPE", "YEAR", "MONTH"])
    .agg(pl.sum("Rainfall (inches)"), pl.sum("AMOUNT"))
    .sort(["YEAR", "MONTH"])
    .collect(engine=gpu_engine)
)

res

shape: (1_584, 5)
┌─────────────────────┬──────┬───────┬───────────────────┬───────────┐
│ EXP_TYPE            ┆ YEAR ┆ MONTH ┆ Rainfall (inches) ┆ AMOUNT    │
│ ---                 ┆ ---  ┆ ---   ┆ ---               ┆ ---       │
│ str                 ┆ i64  ┆ i64   ┆ f64               ┆ f64       │
╞═════════════════════╪══════╪═══════╪═══════════════════╪═══════════╡
│ Health              ┆ 2010 ┆ 1     ┆ 51.45             ┆ 12852.68  │
│ Groceries           ┆ 2010 ┆ 1     ┆ 294.3             ┆ 26341.02  │
│ Bills and Utilities ┆ 2010 ┆ 1     ┆ 59.41             ┆ 26307.97  │
│ Fines               ┆ 2010 ┆ 1     ┆ 0.91              ┆ 315.2     │
│ Education           ┆ 2010 ┆ 1     ┆ 25.67             ┆ 20602.49  │
│ …                   ┆ …    ┆ …     ┆ …                 ┆ …         │
│ Fines               ┆ 2020 ┆ 12    ┆ 3.43              ┆ 13707.66  │
│ Clothing            ┆ 2020 ┆ 12    ┆ 413.24            ┆ 5.5658e6  │
│ Motor/Travel        ┆ 2020 ┆ 12    ┆ 1434.55           ┆ 1.5570e7  │
│ Groceries           ┆ 2020 ┆ 12    ┆ 2246.48           ┆ 4.2694e6  │
│ Savings             ┆ 2020 ┆ 12    ┆ 129.21            ┆ 559804.01 │
└─────────────────────┴──────┴───────┴───────────────────┴───────────┘

In [ ]:
(
    res
    .with_columns(
        pl.date(pl.col("YEAR"), pl.col("MONTH"), 1).alias("date-month"),
        pl.col("Rainfall (inches)")*100,
    )
    .hvplot.line(
        x="date-month", y=["AMOUNT", "Rainfall (inches)"],
        by=['EXP_TYPE'],
        rot=45,
    )
)

:NdOverlay   [Variable]
   :Curve   [date-month]   (value)

In [ ]:
engine_with_fallback = pl.GPUEngine(
    device=0, # This is the default
    raise_on_fail=False, # Fallback to CPU if we can't run on the GPU (this is the default)
)

In [ ]:
%%time

result = transactions.with_columns(
    pl.col('AMOUNT').rolling_mean(
        window_size=7
    )
     .alias('rolling_avg')
).collect(engine=engine_with_fallback)

result.head()

CPU times: user 8.3 s, sys: 3.49 s, total: 11.8 s
Wall time: 7.47 s


shape: (5, 11)
┌────────────┬────────────┬──────────┬──────────────┬───┬─────┬─────────────┬────────┬─────────────┐
│ CUST_ID    ┆ START_DATE ┆ END_DATE ┆ TRANS_ID     ┆ … ┆ DAY ┆ EXP_TYPE    ┆ AMOUNT ┆ rolling_avg │
│ ---        ┆ ---        ┆ ---      ┆ ---          ┆   ┆ --- ┆ ---         ┆ ---    ┆ ---         │
│ str        ┆ date       ┆ date     ┆ str          ┆   ┆ i64 ┆ str         ┆ f64    ┆ f64         │
╞════════════╪════════════╪══════════╪══════════════╪═══╪═════╪═════════════╪════════╪═════════════╡
│ CI6XLYUMQK ┆ 2015-05-01 ┆ null     ┆ T8I9ZB5A6X90 ┆ … ┆ 11  ┆ Motor/Trave ┆ 20.27  ┆ null        │
│            ┆            ┆          ┆ UG8          ┆   ┆     ┆ l           ┆        ┆             │
│ CI6XLYUMQK ┆ 2015-05-01 ┆ null     ┆ TZ4JSLS7SC7F ┆ … ┆ 8   ┆ Motor/Trave ┆ 12.85  ┆ null        │
│            ┆            ┆          ┆ O9H          ┆   ┆     ┆ l           ┆        ┆             │
│ CI6XLYUMQK ┆ 2015-05-01 ┆ null     ┆ TTUKRDDJ6B6F ┆ … ┆ 1   ┆ Housing     ┆ 383.8  ┆ null        │
│            ┆            ┆          ┆ 42H          ┆   ┆     ┆             ┆        ┆             │
│ CI6XLYUMQK ┆ 2015-05-01 ┆ null     ┆ TDUHFRUKGPPI ┆ … ┆ 16  ┆ Entertainme ┆ 5.72   ┆ null        │
│            ┆            ┆          ┆ 6HD          ┆   ┆     ┆ nt          ┆        ┆             │
│ CI6XLYUMQK ┆ 2015-05-01 ┆ null     ┆ T0JBZHBMSVRF ┆ … ┆ 15  ┆ Entertainme ┆ 11.06  ┆ null        │
│            ┆            ┆          ┆ MMD          ┆   ┆     ┆ nt          ┆        ┆             │
└────────────┴────────────┴──────────┴──────────────┴───┴─────┴─────────────┴────────┴─────────────┘